In [1]:
import os
import matplotlib.pyplot as plt
import time
import string
import random
import pandas as pd
import json
import os.path
import re

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
path = os.path.abspath('../WTO_data_article')

all_urls = set()
for folder in os.listdir(path):
    print(folder)
    sub_urls = set()
    if not '.' in folder:
        for file in os.listdir(path + '/' + folder):
            # print(file)
            data = pd.read_csv(path + '/' + folder + '/' + file)
            sub_urls = sub_urls.union(set(data['url']))
            all_urls = all_urls.union(set(data['url']))
            sub_urls = {x for x in sub_urls if x == x}
        print('category unique:', len(sub_urls), end='\n\n')
all_urls = {x for x in all_urls if x == x}
print('total unique:', len(all_urls))

News_archives
category unique: 9841

.DS_Store
all_article_content.json
fail_record.json
Subject_archives
category unique: 7927

Press_releases
category unique: 999

DG_speeches
category unique: 1943

total unique: 10110


In [59]:
for x in all_urls:
    if 'aspx' in x:
        print(x)

http://gtad.wto.org/project.aspx?prjCode=TC13/6
http://gtad.wto.org/project.aspx?prjCode=ATPC13/2
https://meetings.wto.org/Press/Journalists/Journalists/journalistregistration.aspx?Language=E
http://stat.wto.org/Home/WSDBHome.aspx
https://meetings.wto.org/NGO/PreRegistration/Preregistration/ngohome.aspx?Language=E


In [17]:
def check_url_postfix(folder):
    df = pd.DataFrame(columns = ['url', 'postfix'])
    for file in os.listdir(path + '/' + folder):
        data = pd.read_csv(path + '/' + folder + '/' + file)
        data = data.dropna()
        temp = pd.DataFrame()
        temp['url'] = data['url']
        temp['postfix'] = [url.split('.')[-1] for url in temp['url']]
        df = pd.concat([df, temp])
    df_gp = df.groupby(by='postfix', as_index=False).size().sort_values('size', ascending=False)
    df_gp['prop'] = df_gp['size']/len(df)
    return df_gp
    # print(df.groupby(by='postfix', as_index=False).size().sum())

In [18]:
check_url_postfix('News_archives')

,postfix,size,prop
13,htm,9173,0.848174
97,mp3,619,0.057235
12,doc,454,0.041979
102,pdf,381,0.035229
106,wmv,48,0.004438
...,...,...,...
43,htm#ds176,1,0.000092
40,htm#ds156,1,0.000092
34,htm#ds108,1,0.000092
33,htm#ds103,1,0.000092


In [19]:
check_url_postfix('Press_releases')

,postfix,size,prop
0,htm,967,0.967
1,mp3,18,0.018
3,wmv,11,0.011
2,pdf,4,0.004


In [21]:
check_url_postfix('Subject_archives')

,postfix,size,prop
8,htm,16442,0.901425
19,mp3,1306,0.071601
7,doc,213,0.011678
24,pdf,158,0.008662
28,wmv,62,0.003399
29,xls,18,0.000987
12,htm#follow_up,4,0.000219
3,aspx,3,0.000164
18,html,3,0.000164
6,aspx?prjCode=TC13/6,3,0.000164


In [20]:
check_url_postfix('DG_speeches')

,postfix,size,prop
1,htm,1679,0.859703
3,mp3,217,0.111111
8,wmv,40,0.020481
0,doc,12,0.006144
2,htm#follow_up,1,0.000512
4,org/en/,1,0.000512
5,org/jag2007,1,0.000512
6,pdf,1,0.000512
7,uk/resources/en/news/15766232/communique-020409,1,0.000512


In [26]:
def check_htm_category(folder):
    number_pattern = r'[0-9]'
    df = pd.DataFrame(columns = ['url', 'htm_category'])
    for file in os.listdir(path + '/' + folder):
        data = pd.read_csv(path + '/' + folder + '/' + file)
        data = data.dropna()
        temp = pd.DataFrame()
        temp['url'] = [url for url in data['url'] if 'htm' in url]
        temp['htm_category'] = [re.sub(number_pattern, '', url.split('/')[-2].split('_')[0]) for url in temp['url']]
        df = pd.concat([df, temp])
    df_gp = df.groupby(by='htm_category', as_index=False).size().sort_values('size', ascending=False)
    df_gp['prop'] = df_gp['size']/len(df)
    return df_gp

In [27]:
check_htm_category('News_archives')

,htm_category,size,prop
34,news,5677,0.611285
56,tpr,925,0.099602
36,pres,779,0.083881
48,spra,302,0.032519
47,sppl,293,0.031549
...,...,...,...
26,forms.wto.org,1,0.000108
28,inftec,1,0.000108
29,its,1,0.000108
37,private,1,0.000108


In [28]:
check_htm_category('Press_releases')

,htm_category,size,prop
5,pres,945,0.977249
4,news,6,0.006205
9,sppl,4,0.004137
1,countries,2,0.002068
11,teccop,2,0.002068
0,agric,1,0.001034
2,dispu,1,0.001034
3,markacc,1,0.001034
6,region,1,0.001034
7,reser,1,0.001034


In [30]:
check_htm_category('Subject_archives').head(5)

,htm_category,size,prop
25,news,11062,0.672053
43,tpr,1590,0.096598
27,pres,1094,0.066464
37,spra,732,0.044471
12,countries,473,0.028736


In [31]:
check_htm_category('DG_speeches').head(5)

,htm_category,size,prop
9,news,754,0.448810
17,spra,304,0.180952
16,sppl,292,0.173810
14,spmm,90,0.053571
8,gcounc,55,0.032738


# url distribution

In [4]:
print('unique urls:', len(all_urls))

unique urls: 10110


In [5]:
htm = 0
for url in all_urls:
    if 'htm' in url:
        htm += 1
print('urls with htm:', htm)
print('urls not with htm:', len(all_urls) - htm)

urls with htm: 8748
urls not with htm: 1362


# detele solved recorded

In [6]:
# read records
with open('../WTO_data_article/fail_record.json') as f:
    fail_record = json.load(f)

with open('../WTO_data_article/all_article_content.json') as f:
    all_article_content = json.load(f)

In [7]:
print(len(all_article_content.keys()))

8680


In [8]:
print(len(fail_record))
fail_record_keys = set(fail_record.keys())
for k_fail in fail_record_keys:
    if k_fail in all_article_content.keys():
        fail_record.pop(k_fail)

with open(f'../WTO_data_article/fail_record.json', 'w') as fp:
    json.dump(fail_record, fp)
    
print(len(fail_record))

68
68


In [49]:
set(fail_record.values())

{'Message: Reached error page: about:neterror?e=dnsNotFound&u=http%3A//forms.wto.org/survey_e.htm&c=UTF-8&d=We%20can%E2%80%99t%20connect%20to%20the%20server%20at%20forms.wto.org.\n',
 'Message: Unable to locate element: .centerCol\n',
 'Message: Unable to locate element: .introTextDiv\n',
 "name 'date' is not defined"}

In [50]:
fail_record_df = pd.DataFrame(fail_record.items(), columns = ['url', 'error'])

In [51]:
look = fail_record_df[fail_record_df.error == "name 'date' is not defined"]
look.shape

(1, 2)

In [58]:
fail_record

{'https://www.wto.org/english/tratop_e/tpr_e/tp509_crc_e.htm': 'Message: Unable to locate element: .centerCol\n',
 'https://www.wto.org/english/news_e/news02_e/dsb_11nov02_e.htm': 'Message: Unable to locate element: .introTextDiv\n',
 'https://www.wto.org/english/news_e/news02_e/dsb_07june02_e.htm': 'Message: Unable to locate element: .introTextDiv\n',
 'https://www.wto.org/english/news_e/tratop_e/dispu_e/dispu_e.htm': 'Message: Unable to locate element: .centerCol\n',
 'https://www.wto.org/english/forums_e/public_forum_e/session_26_num23_e.htm': 'Message: Unable to locate element: .introTextDiv\n',
 'https://www.wto.org/english/forums_e/public_forum_e/session_25_num3_e.htm': 'Message: Unable to locate element: .introTextDiv\n',
 'https://www.wto.org/english/res_e/reser_e/econsem_e.htm': 'Message: Unable to locate element: .centerCol\n',
 'https://www.wto.org/english/forums_e/public_forum_e/session_26_num28_e.htm': 'Message: Unable to locate element: .introTextDiv\n',
 'https://www.wto

In [91]:
for x in look.url:
    print(x)

https://www.wto.org/english/news_e/news02_e/dsb_11nov02_e.htm
https://www.wto.org/english/news_e/news02_e/dsb_07june02_e.htm
https://www.wto.org/english/forums_e/public_forum_e/session_26_num23_e.htm
https://www.wto.org/english/forums_e/public_forum_e/session_25_num3_e.htm
https://www.wto.org/english/forums_e/public_forum_e/session_26_num28_e.htm
https://www.wto.org/english/forums_e/public_forum_e/session_26_num31_e.htm
https://www.wto.org/english/forums_e/public_forum_e/session_25_num10_e.htm
http://www.wto.org/english/forums_e/public_forum_e/forum06_e.htm
https://www.wto.org/english/forums_e/public_forum_e/session_26_num36_e.htm
https://etraining.wto.org/Take%20a%20Tour/player.html
https://www.wto.org/english/news_e/news10_e/wtr_11nov_e.htm
https://www.wto.org/english/forums_e/public_forum_e/session_26_num30_e.htm
https://www.wto.org/english/forums_e/public_forum_e/session_26_num17_e.htm
https://www.wto.org/english/forums_e/public_forum_e/session_26_num21_e.htm
http://www.dfa.gov.za/

In [43]:
cate = set()
for x in look.url:
    print(x)
    cate.add(x.split('/')[-2][:-2])

https://www.wto.org/english/tratop_e/tpr_e/tp509_crc_e.htm
https://www.wto.org/english/news_e/tratop_e/dispu_e/dispu_e.htm
https://www.wto.org/english/res_e/reser_e/econsem_e.htm
https://www.wto.org/english/tratop_e/envir_e/te035_e.htm
https://www.wto.org/english/tratop_e/servte_e/gbtoff.htm
https://www.wto.org/english/news_e/news98_e/231198.htm
https://www.wto.org/english/tratop_e/tpr_e/tp439_e.htm
http://www.wto.org/wto/speeches/NY.htm
https://www.wto.org/english/thewto_e/whatis_e/tif_e/org4_e.htm
https://www.wto.org/english/thewto_e/vacan_e/fast_track_e.htm
https://www.wto.org/english/tratop_e/envir_e/te036_e.htm
https://www.wto.org/tratop_e/dda_e/dda_e.htm
https://www.wto.org/english/tratop_e/envir_e/te033_e.htm
https://www.wto.org/english/news_e/news02_e/fact_sheettech_coop_e.htm
https://www.wto.org/english/thewto_e/whatis_e/tif_e/org6_e.htm
https://www.wto.org/english/news_e/news08_e/dsb_23sep08_e.htm


In [ ]:
news, pres, wkshop